In [27]:
from urllib.request import urlopen
import pandas as pd
import time
import re
import tweepy
# import modules.settings as Settings
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime

# 曜日（英語 ⇒ 日本語）
WEEK_DAYS_DICT: dict = MappingProxyType({
        'Mon':'（月）',
        'Tue':'（火）',
        'Wed':'（水）',
        'Thu':'（木）',
        'Fri':'（金）',
        'Sat':'（土）',
        'Sun':'（日）'
})
    
# 1ツイートに付与する試合情報数
MAX_MATCH_INFO = 5


def scrapeSportsNaviPage(search_word: str):

    # 対象ページ
    url = 'https://soccer.yahoo.co.jp/ws/category/eng'

    # スクレイピング
    html = urlopen(url).read().decode('utf-8')

    df = pd.read_html(html)[0]

    print(search_word)

    # 検索日付に絞る
    df = df[df['日時'].str.startswith(search_word)]

    return df


# 現在の日付と曜日を取得
today = datetime.date.today().strftime("%Y%m%d")
day_of_week = datetime.date.today().strftime("%a")

# 昨日の日付と曜日を取得（2023/03時点での読み込み時間が17時なので現状本ソースは不要かと思います）
    # yesterday = datetime.date.today() - datetime.timedelta(days=1)
    # day_of_week_yesterday = yesterday.strftime("%a")

# 検索キーを作成
ja_day_of_week = WEEK_DAYS_DICT.get(day_of_week)
t_search_key = str(int(today[4:6])) + '/' + str(int(today[6:8])) + ja_day_of_week

In [29]:
df2 = scrapeSportsNaviPage(t_search_key)
df2

4/1（土）


,日時,種別,ホーム,スコア,アウェイ
0,4/1（土） 20:30,第29節,マンチェスター・C,4 - 1 試合終了,リヴァプール
1,4/1（土） 23:00,第29節,ボーンマス,- スタメン,フラム
2,4/1（土） 23:00,第29節,アーセナル,- スタメン,リーズ
3,4/1（土） 23:00,第29節,ブライトン,- スタメン,ブレントフォード
4,4/1（土） 23:00,第29節,クリスタル・パレス,- スタメン,レスター
5,4/1（土） 23:00,第29節,フォレスト,- スタメン,ウルヴァーハンプトン
6,4/1（土） 25:30,第29節,チェルシー,- 対戦データ,アストン・ヴィラ


In [30]:
# スコア列の値が「試合中止」である行を削除
df3 = df2[df2['スコア'] != '- 試合中止']

# 削除後のデータを表示
df3

,日時,種別,ホーム,スコア,アウェイ
0,4/1（土） 20:30,第29節,マンチェスター・C,4 - 1 試合終了,リヴァプール
1,4/1（土） 23:00,第29節,ボーンマス,- スタメン,フラム
2,4/1（土） 23:00,第29節,アーセナル,- スタメン,リーズ
3,4/1（土） 23:00,第29節,ブライトン,- スタメン,ブレントフォード
4,4/1（土） 23:00,第29節,クリスタル・パレス,- スタメン,レスター
5,4/1（土） 23:00,第29節,フォレスト,- スタメン,ウルヴァーハンプトン
6,4/1（土） 25:30,第29節,チェルシー,- 対戦データ,アストン・ヴィラ


In [31]:
# スコア列の値が「試合中止」または「試合終了」である行を削除
df4 = df2[~df2['スコア'].str.contains('試合中止|試合終了')]
df4

,日時,種別,ホーム,スコア,アウェイ
1,4/1（土） 23:00,第29節,ボーンマス,- スタメン,フラム
2,4/1（土） 23:00,第29節,アーセナル,- スタメン,リーズ
3,4/1（土） 23:00,第29節,ブライトン,- スタメン,ブレントフォード
4,4/1（土） 23:00,第29節,クリスタル・パレス,- スタメン,レスター
5,4/1（土） 23:00,第29節,フォレスト,- スタメン,ウルヴァーハンプトン
6,4/1（土） 25:30,第29節,チェルシー,- 対戦データ,アストン・ヴィラ
